In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import gc

from utils.schemas import *
from utils.functions import *

In [2]:
data_folder = 'input'

In [3]:
train = pd.read_csv(data_folder+'/train_generated_1.csv', dtype = schema_generated_1)
test = pd.read_csv(data_folder+'/test_generated_1.csv', dtype = schema_generated_1)

In [4]:
train.shape, test.shape

((590540, 558), (506691, 558))

In [5]:
train_index = train.shape[0]
test_index = test.shape[0]
df = pd.concat([train, test], axis=0)
df.shape

(1097231, 558)

In [6]:
old_cols = df.columns

In [7]:
del train
del test
gc.collect()

7

In [14]:
df.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_38_fe2,R_emaildomain_0_fe2,device_version_fe2,device_name_fe2,P_emaildomain_0_fe2,P_emaildomain_1_fe2,proc_id_30_0_fe2,proc_id_30_1_fe2,R_emaildomain_1_fe2,proc_id_31_0_fe2
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,2,1,0,0,0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,8,0,0,0,0,0
3,2.0,0.0,1.0,0.0,25.0,1.0,5.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,2,0,2,3,0,0,7,4,0,8


In [19]:
# %%time
# df.groupby(['card1','C1','addr1','id_01','V258','D1']).transform('std')['TransactionAmt']

# result:
#     CPU times: user 9.88 s, sys: 4.71 s, total: 14.6 s
#     Wall time: 14.6 s

In [20]:
# %%time
# df.groupby(['card1','C1','addr1','id_01','V258','D1'])['TransactionAmt'].transform('std')

# result:
#     CPU times: user 158 ms, sys: 397 µs, total: 159 ms
#     Wall time: 158 ms

In [21]:
df['Transaction_day_of_week'] = np.floor((df['TransactionDT'] / (3600 * 24) - 1) % 7)
df['Transaction_hour'] = np.floor(df['TransactionDT'] / 3600) % 24

In [22]:
cols1 = [x for x in df.columns if 'card' in x and 'fe' not in x]
df['N1'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [9]:
cols1 = [x for x in df.columns if 'addr' in x]
df['N2'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [10]:
cols1 = [x for x in df.columns if 'C' in x and 'Product' not in x]
df['N3'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [11]:
cols1 = [x for x in df.columns if 'mail' in x]
df['N4'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [ ]:
cols1 = ['P_emaildomain', 'R_emaildomain', 'addr1']
df['N5'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [ ]:
cols_ex = ['DeviceInfo','DeviceType','TransactionDT','DeviceType_fe1','DeviceInfo_fe1','DeviceType_fe2','DeviceInfo_fe2','ProductCD','ProductCD_fe1','ProductCD_fe2']
cols1 = [x for x in df.columns if 'D' in x and x not in cols_ex]
df['N6'] = df.groupby(cols1)['TransactionDT'].transform('count')

In [30]:
# df = df.drop(['N1', 'N2', 'N3', 'N4'], axis=1)

In [31]:
# for c in ['N1', 'N2', 'N3', 'N4']:
#     df = freq_encoder(df, c, c, 0.005)

In [32]:
# from sklearn.cluster import KMeans

In [33]:
# n_cls = [2, 4, 8, 16, 32, 64]

In [34]:
# cols1 = [x for x in df.columns if 'C' in x and 'Product' not in x]
# cols2 = [x for x in df.columns if 'card' in x and 'fe' not in x]
# cols3 = [x for x in df.columns if 'V' in x]

# cols_group = [cols1, cols2, cols3]

In [35]:
# for n in n_cls:
#     m = 1
#     for cols in cols_group:
#         print('Generation feature:\tgroup_{0}_n_{1}'.format(m, n))
#         kmeans = KMeans(n_clusters=n, random_state=42).fit(df[cols].fillna(-1))
#         df['group_{0}_n_{1}'.format(m, n)] = kmeans.labels_
#         m += 1

In [12]:
df['C13_vs_C1'] = np.log1p(df.C13/df.C1)

In [13]:
df['D1_vs_D15'] = np.log1p(df.D1/df.D15)

In [14]:
df['TransactionAmt_vs_mean'] = np.log1p(df.TransactionAmt/df.TransactionAmt.values.mean())

In [16]:
df['N3_vs_N1'] = np.log1p(df.N3/df.N1)

In [5]:
# train = pd.read_csv(data_folder+'/train_ft_eng_0.csv', dtype = schema_generated_0)
# test = pd.read_csv(data_folder+'/test_ft_eng_0.csv', dtype = schema_generated_0)

In [17]:
train = df.iloc[:train_index, :]
test = df.iloc[train_index:, :]

In [18]:
train.to_csv(data_folder+'/train_ft_eng_1.csv', header=True, index=None)
print('train guardado')
test.to_csv(data_folder+'/test_ft_eng_1.csv', header=True, index=None)
print('test guardado')

train guardado
test guardado


In [6]:
train.shape, test.shape

((590540, 576), (506691, 576))

In [7]:
train.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,group_3_n_8,group_1_n_16,group_2_n_16,group_3_n_16,group_1_n_32,group_2_n_32,group_3_n_32,group_1_n_64,group_2_n_64,group_3_n_64
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,6,0,0,16,0,0,49,3
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,18,0,0,25,3
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,18,0,0,25,3
3,2.0,0.0,1.0,0.0,25.0,1.0,5.0,0.0,0.0,0.0,...,0,0,0,0,22,0,0,33,17,3
4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,4,0,2,15,0,15,26,0,45,15


In [8]:
train_index = train.shape[0]
test_index = test.shape[0]
df = pd.concat([train, test], axis=0)
df.shape

(1097231, 576)